In [8]:
import re
from collections import Counter
import scrabble_assistant as sa
import dictionary_preprocessing as dp

# Выкидываем неподходящие слова из словаря

In [2]:
dp.drop_incorrect_words()

# Разобьем исходый словарь на подсловари

In [3]:
dp.make_sub_dictionaries()

# Считаем словарь

In [44]:
words = []

with open(sa.DICTIONARY_FILENAME, 'r', encoding='utf-8') as dict_file:
    for line in dict_file:
        word = line[:-2]
        words.append(word)
words

абажу
аба
абази
абазинк
абак
абба
аббатис
аббатисс
аббатств
аббревиатур
абве
абверове
аберраци
абза
абиогене
абиссал
абиссине
абиссинк
абитуриен
абитуриентк
аболициониз
аболиционис
аболиционистк
абонемен
абонен
аборда
аборигенк
абор
абрази
абрази
абракадабр
абре
абрико
абрикоти
абри
абсен
абсентеиз
абсолю
абсолютизаци
абсолютиз
абсолютис
абсолютистк
абсолютност
абсорбен
абсорбци
абстрагировани
абстрак
абстрактност
абстракциониз
абстракционис
абстракци
абсур
абсурдност
абсцес
абсцисс
абули
абха
абхазе
абхазк
абцу
авангар
авангардиз
авангардис
аванза
аванлож
аванпор
аванпос
аван
авансировани
авансодател
авансодержател
авансцен
авантюр
авантюриз
авантюри
авантюрис
авантюристк
авантюрност
аваре
аварийност
авари
аварк
авги
авгу
авгус
августине
авиабаз
авиабиле
авиабомб
авиагоризон
авиадесан
авиадесантни
авиаконструкто
авиалайне
авиалини
авиамагистрал
авиаматк
авиамехани
авиамоделиз
авиамоделис
авиамодел
авиамото
авианосе
авиапассажи
авиапочт
авиасало
авиаспор
авиастроени
авиастроител
авиато

булл
булочна
булочни
булочниц
булыжни
бульва
бульдозе
бульдозерис
булькани
бульо
бумазе
бумеран
бун
бундесве
бундесра
бундеста
бундове
бунке
бункеровк
бунтарств
бунтар
бунтовщи
бунтовщиц
бунчужны
бунчу
бур
бура
бура
бура
бур
бургонско
бурдю
бурева
буревестни
бурело
буреломни
бурени
буржуази
буржуазност
бурильщи
бурла
бурлачеств
бурлени
бурлес
бурлеск
бурливост
бурмист
бурну
бурозубк
бурс
бурса
бурси
бур
буртовани
буртоукладчи
буру
бурунду
бурундучоно
бурчани
бурья
бур
буря
бурятк
бусин
буссол
бусте
бу
бутадие
бутар
бутафо
бутафори
бутафорска
бутербро
бутиромет
буто
бутонизаци
бутоньерк
бутс
бутылк
бутылконо
бутыл
буфе
буфе
буфетчи
буфетчиц
бухани
буханк
бухарни
бухгалте
бухгалтери
бушевани
бушел
бушла
бушме
бушменк
бушпри
бывалост
быдл
былинк
было
был
быльни
быстродействи
быстрот
быстротечност
быстрото
быстроходност
быстрянк
бы
бытност
бытовани
бытовиз
бытописани
бытописател
бычатин
бычин
бье
бэ
бюва
бюве
бюдже
бюкс
бюрге
бюргерств
бюретк
бюрокра
бюрократизаци
бюрократиз
бюрократи
бюс


гидрохими
гидроэнергетик
гидроэнерги
гильди
гильз
гильотин
гим
гимназис
гимназистк
гимнази
гимнас
гимнастик
гимнастк
гинеке
гинеколо
гинекологи
гине
гиперболизаци
гиперболиз
гиперболичност
гиперболои
гиперборее
гиперборейк
гипереми
гиперзву
гиперо
гипертерми
гипертони
гипертрофи
гипно
гипноло
гипнотерапи
гипнотизаци
гипнотиз
гиповитамино
гиподинами
гипокинези
гипокси
гипотез
гипотенуз
гипотерми
гипотетичност
гипотони
гипофи
гипоцент
гиппопота
гип
гипсовани
гипсометри
гипю
гирл
гирлянд
гирокомпа
гироско
гир
гистоло
гистологи
ги
гита
гитан
гитар
гитарис
гитаристк
гитлериз
гитлерове
гито
гиф
гичк
главар
главенств
глав
главко
главпочтам
главре
главстаршин
глаго
глаголиц
глагол
глагольност
гладиато
гладильна
гладильщи
гладильщиц
гладиолу
гладкост
глажени
гла
глазе
глазировани
глазировк
глазировщи
глазировщиц
глазищ
глазниц
глазоме
глазунь
глазуровани
глазуровк
глазур
гланд
гла
гласност
глауком
глашата
глетче
гликоге
глин
глинистост
глинищ
глиномешалк
глиномя
глиномялк
глинтвей
глиптик
глипт

закреплени
закрепощени
закро
закройк
закройна
закройщи
закройщиц
закро
закруглени
закрутк
закручивани
закрывани
закрыти
заку
закупани
закупк
закупоривани
закупорк
закупорщи
закупорщиц
закупщи
закупщиц
закуривани
закуск
закусочна
закусывани
заку
закутывани
за
зал
залавливани
залакировывани
заламывани
залегани
заледенени
залежалост
залеж
залезани
залесь
залечивани
заливани
заливистост
заливк
заливно
заливчатост
заливщи
заливщиц
зализ
зализывани
залогодател
залогодержател
заложени
заложни
заложниц
зал
залужени
залысин
зальц
замазк
замазывани
замаливани
заманивани
заманих
заманчивост
замариновывани
замасленност
замасливани
заматывани
зама
замачивани
замащивани
замедлени
замедленност
замедлител
замеливани
замен
заменимост
заменител
заменяемост
заме
замерзаемост
замерзани
замерщи
заме
заместител
заметани
заметк
заметност
замечани
замечательност
замешательств
замещаемост
замещени
замирани
замкнутост
замокани
замолачивани
замо
замораживани
заморозо
замочни
замощени
замполи
замужеств
замуровыва

кител
китобо
кифар
кифо
кичливост
кишени
кишечни
кишк
кишла
кишлачни
киянк
клавеси
клавиатур
клавикор
клави
клавираусцу
клави
клавиш
кла
кладк
кладова
кладовщи
кладовщиц
кладоискател
кладчи
клак
кла
клапа
клапшто
кларе
кларне
кларнетис
кларнетистк
классик
классификаци
классичност
классна
классовост
клаузул
клевани
клеве
клеверищ
клевет
клеветни
клеветниц
клево
клевре
клеева
клееварени
клеени
кле
клейковин
клеймени
клейм
клеймовщи
клейсте
клемм
клепал
клепальщи
клепальщиц
клепани
клеппе
клепсидр
клептокра
клептократи
клептократк
клептома
клептомани
клептоманк
клерика
клерикализ
клеточни
клеточниц
клетчатк
клет
клешн
кле
клещевин
клещевинни
клещеножк
кливе
клиен
клиентел
клиентк
клиентур
клизм
кли
клик
кликушеств
климактери
клима
климатографи
климатоло
климатологи
климатотерапи
кли
клиник
клиницис
клинке
клинкеровани
клинке
клино
клиномет
клинопис
клинту
кли
клипе
клип
клипс
кли
клири
клирин
клиро
клирошани
клирошанк
кли
кличк
клишировани
клоак
клобу
клоко
клокотани
клокту
кло
клонирован

...

приплющивани
приплясывани
приподнимани
приподняти
приподнятост
припо
припойк
приползани
приполо
припоминани
припорашивани
припоро
припорошк
припосадк
приправ
приправк
приправлени
припрыжк
припрыскивани
припрягани
припрядывани
припряжк
припрятани
припрятывани
припугивани
припудривани
припус
припускани
припутывани
припухани
припы
припыливани
прирабатывани
приработк
приработо
прирастани
приращени
приращивани
прире
прирезани
прирезк
прирезывани
прирез
приречь
природ
природове
природоведени
природолю
прирос
приросто
приру
прирубани
прирубк
прируливани
приручени
присад
присадо
присаливани
присасывани
присборивани
присваивани
присвис
присвистк
присвистывани
присвоени
присвоител
присвоительниц
присворивани
присе
присевани
присе
приседани
присекани
приселени
присеменни
присес
присечк
присказк
присказывани
прискакивани
приско
прискорби
прискочк
присловь
прислуг
прислуживани
прислужни
прислужничеств
прислушивани
присмаливани
присматривани
присмот
приснащивани
присовокуплени
присоединени
присо
при

резьбови
резьбовщи
резьбовщиц
резюмировани
реинфекци
ре
рейде
рейдировани
рейк
рейнвей
рей
рейсму
рейсфеде
рейсшин
рейта
рейте
рейтин
рейхсве
рейхсканцле
рейхсра
рейхста
рек
реквие
реквизи
реквизито
реклам
рекламаци
рекламировани
рекламодател
рековедени
рекогносцировк
рекомендаци
рекомендующи
реконверси
реконструкци
рекор
рекордиз
рекордис
рекордистк
рекордсме
рекордсменк
рекордсменств
рекоста
рекреаци
рекредити
рекру
рекрутств
рекрутчин
ректифика
ректификаци
ректо
ректора
рекуперато
рекупераци
религиозност
религи
реликви
релик
релье
рельефност
рель
рельсопрокатчи
релятивиз
релятивис
релятивно
реляци
ремарк
ремедиу
реме
ременщи
ремен
ремесл
ремешо
ремилитаризаци
ремне
ремнецветни
ремонтантност
ремонтировщи
ремонтни
ренат
ренега
ренегатств
ренессан
рене
рени
ренкло
ренон
рент
рентабельност
рентге
рентгенизаци
рентгенограмм
рентгенографи
рентгеноло
рентгенологи
рентгеноскопи
рентгенотерапи
реомет
реорганизаци
реоста
реп
репатриан
репатриантк
репатриаци
репатриируема
репатриируемы
репейни

слободчани
слободчанк
слобожани
слобожанк
слова
словар
словачк
словене
словенк
словесни
словесниц
словесност
словине
словинк
словни
словоблу
словоблуди
словобоязн
словоговорени
словоизвержени
словоизлияни
словоизменени
словолитчи
словообили
словопрени
словосложени
словосочетани
словотворчеств
словотолковани
слогоразде
слоевищ
слоени
сложени
сложност
слоистост
сло
слойк
сло
слоних
слоновни
слоновост
слоновщи
слуг
служанк
служаща
служебни
служебност
служител
служительниц
слу
слуха
случа
случайно
случайност
случившеес
случк
слушани
слушател
слушательниц
слушающа
слышанно
слышимо
слышимост
слюд
слюдини
слюн
слюного
слюноотделени
слюнотечени
сля
слябин
смазк
смазчи
смазчиц
смазывани
смаковани
смале
смальт
сматывани
смахивани
смачиваемост
смачивани
смачивател
смежност
смела
смелост
смен
сменщи
сменщиц
сменяемост
смер
смердени
смердяковщин
смерзани
смертельност
смертна
смертност
смертоносност
смер
смесител
смеск
смес
сметан
сметани
сметчи
смехотворност
смехотворств
смешанност
смешени
смешивае

трапезар
трапезни
трапезниц
трапецои
трапецоэд
трап
траппе
траппис
трас
трасс
трассан
трасса
трат
тратт
траттори
трауле
трау
траурниц
трафаре
трафаретк
трафаретност
трафаретчи
трафлени
трахеи
трахеотоми
трахе
трахи
трахом
требищ
требни
требовани
требух
тревог
тревожност
трезва
трезвенни
трезвенниц
трезвенност
трезво
трезвост
трезвучи
трезубе
тре
трекратност
трембит
тремоли
тремо
тре
трена
трене
трензельк
трени
тренин
тренированност
тренировк
треножни
тренчи
тренькани
трепа
трепал
трепальн
трепальщи
трепальщиц
трепа
трепанаци
трепан
трепе
трепе
трепетани
трепетност
трес
трес
трест
трестировани
третировани
трет
третьеклассни
третьеклассниц
третьекурсни
третьекурсниц
треуголк
трефно
трешко
трешкоу
трещин
трещиноватост
триангуляци
триа
триб
трибу
трибун
трибуна
тривиальност
тривиу
тригонометри
тридцатилети
триединств
трие
триер
триеровани
триеровк
тризн
трикири
триклини
трикота
трикоти
триктра
трилистни
трилле
трилоби
трилоги
тримест
тринадцатилети
тринитротолуо
трино
трио
триод
триоле
три

целин
целинни
целительност
целковы
целлофа
целлулои
целлюли
целовани
целомудренност
целомудри
целоста
целостност
целост
цел
цельност
цемен
цементировани
цементировк
цементи
цементни
цементово
цемянк
цен
цен
цензо
цензорств
цензур
цензурност
цензуровани
ценител
ценни
ценно
ценност
ценообразовани
цен
центне
централиз
централ
центральност
центрировани
центрифуг
центричност
центровани
центровщи
центропла
центросом
центурио
центури
цеоли
це
цепенени
цепен
цепкост
цепочк
цеппели
цеп
цербе
церемониа
церемони
церемонност
церковно
церковност
церков
цесареви
цесаревн
цесарк
цехи
цианировани
циано
цивилис
цивилистик
цигейк
цикад
цик
цикламе
циклевани
цикличност
цикловани
циклоид
цикло
циклотро
цикл
цикори
цикут
цимбалис
цимлянско
цинг
цинерари
циничност
цин
цинки
цинковани
цинкогра
цинкографи
цинни
циновк
цинубел
циперу
цирко
циркони
циркорам
циркулировани
циркул
циркуля
цирро
цисти
цитадел
цитат
цитатничеств
цитировани
цитогенетик
цитологи
цитоплазм
цитохими
цитр
цитру
цитрусово
цитрусоводств
ци

# Считаем расположение бонусов

In [46]:
best_hint = sa.read_json_to_list(sa.BOARD_BONUSES_FILENAME)
start_row = best_hint[int(len(best_hint) / 2)]
start_row = start_row[1:-1]
start_row

['00', '00', 'x2', '00', '00', '00', 'ST', '00', '00', '00', 'x2', '00', '00']

In [30]:
# pattern = re.compile('[а-я]{,6}[а-я][а-я]{,6}')

# Найдем наилучший первый ход:

In [60]:
row = ['', '', '', 'с', 'у', 'п', '', '']
word_start_index = [i for i in range(len(row)) if row[i]][0]
word_start_index

3